In [1]:
# Notebook support or argpase
import sys; sys.argv=['']; del sys

In [2]:
%matplotlib inline
import matplotlib.pyplot as plt

import os
import time
import random
import numpy as np
import torch as th
import torch.nn as nn
import torch.nn.functional as F
import apex

from collections import deque
from torchinfo import summary

import tools
from configurator import generate_args, get_arg_dict
from th_logger import TBXLogger as TBLogger

# Env deps: Soundspaces and Habitat
from habitat.datasets import make_dataset
from ss_baselines.av_nav.config import get_config
from ss_baselines.savi.config.default import get_config as get_savi_config
from ss_baselines.common.env_utils import construct_envs
from ss_baselines.common.environments import get_env_class
from ss_baselines.common.utils import images_to_video_with_audio

# Custom ActorCritic agent for PPO
from models import ActorCritic, ActorCritic_DeepEthologyVirtualRodent, \
    Perceiver_GWT_GWWM_ActorCritic, Perceiver_GWT_AttGRU_ActorCritic

# Dataset utils
from torch.utils.data import IterableDataset, DataLoader
import compress_pickle as cpkl

/home/rousslan/anaconda3/envs/ss-hab-headless-py39/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# This variant will sample one single (sub) seuqence of an episode as a trajectoyr
# and add zero paddign to the rest
class BCIterableDataset3(IterableDataset):
    def __init__(self, dataset_path, batch_length, seed=111):
        self.seed = seed
        self.batch_length = batch_length
        self.dataset_path = dataset_path

        # Read episode filenames in the dataset path
        self.ep_filenames = os.listdir(dataset_path)
        if "dataset_statistics.bz2" in self.ep_filenames:
            self.ep_filenames.remove("dataset_statistics.bz2")
        
        print(f"Initialized IterDset with {len(self.ep_filenames)} episodes.")
    
    def __iter__(self):
        batch_length = self.batch_length
        while True:
            # Sample one episode file
            idx = th.randint(len(self.ep_filenames), ())
            ep_filename = self.ep_filenames[idx]
            ep_filepath = os.path.join(self.dataset_path, ep_filename)
            with open(ep_filepath, "rb") as f:
                edd = cpkl.load(f)
            is_success = edd["info_list"][-1]["success"]
            last_action = edd["action_list"][-1]
            print(f"Sampled traj idx: {idx}; Length: {edd['ep_length']}; Success: {is_success}; Last act: {last_action}")
            
            # edd_start = th.randint(0, edd["ep_length"]-20, ()).item() # Sample start of sub-squence for this episode
            # NOTE: the following sampling might not leverage long-term trajectories well.
            edd_start = 0 # Given that we have short trajectories, just start at the beginning anyway
            edd_end = min(edd_start + batch_length, edd["ep_length"])
            subseq_len = edd_end - edd_start
            
            horizon = subseq_len

            obs_list = {
                k: np.zeros([batch_length, *np.shape(v)[1:]]) for k,v in edd["obs_list"].items()
            }
            action_list, reward_list, done_list, depad_mask_list = \
                np.zeros([batch_length, 1]), \
                np.zeros([batch_length, 1]), \
                np.zeros([batch_length, 1]), \
                np.zeros((batch_length, 1)).astype(np.bool8)

            for k, v in edd["obs_list"].items():
                obs_list[k][:horizon] = v[edd_start:edd_end]
            # Adjust the shape of obs_list["depth"] from (T, H, W) -> (T, H, W, 1))
            obs_list["depth"] = obs_list["depth"][:, :, :, None]
            action_list[:horizon] = np.array(edd["action_list"][edd_start:edd_end])[:, None]
            reward_list[:horizon] = np.array(edd["reward_list"][edd_start:edd_end])[:, None]
            done_list[:horizon] = np.array(edd["done_list"][edd_start:edd_end])[:, None]
            depad_mask_list[:horizon] = True

            yield obs_list, action_list, reward_list, done_list, depad_mask_list
    
def make_dataloader3(dataset_path, batch_size, batch_length, seed=111, num_workers=2):
    def worker_init_fn(worker_id):
        # worker_seed = th.initial_seed() % (2 ** 32)
        worker_seed = 133754134 + worker_id

        random.seed(worker_seed)
        np.random.seed(worker_seed)

    th_seed_gen = th.Generator()
    th_seed_gen.manual_seed(133754134 + seed)

    dloader = iter(
        DataLoader(
            BCIterableDataset3(
                dataset_path=dataset_path, batch_length=batch_length),
                batch_size=batch_size, num_workers=num_workers,
                worker_init_fn=worker_init_fn, generator=th_seed_gen
            )
    )

    return dloader

# Tensorize current observation, store to rollout data
def tensorize_obs_dict(obs, device, observations=None, rollout_step=None):
    obs_th = {}
    for obs_field, _ in obs[0].items():
        v_th = th.Tensor(np.array([step_obs[obs_field] for step_obs in obs], dtype=np.float32)).to(device)
        # in SS1.0, the dcepth observations comes as [B, 128, 128, 1, 1], so fix that
        if obs_field == "depth" and v_th.dim() == 5:
            v_th = v_th.squeeze(-1)
        obs_th[obs_field] = v_th
        # Special case when doing the rollout, also stores the 
        if observations is not None:
            observations[obs_field][rollout_step] = v_th
    
    return obs_th

In [4]:
# region: Generating additional hyparams
CUSTOM_ARGS = [
    # General hyper parameters
    get_arg_dict("seed", int, 111),
    get_arg_dict("total-steps", int, 1_000_000),
    
    # Behavior cloning gexperiment config
    get_arg_dict("dataset-path", str, "SAVI_Oracle_Dataset_v0"),

    # SS env config
    get_arg_dict("config-path", str, "env_configs/savi/savi_ss1.yaml"),

    # PPO Hyper parameters
    get_arg_dict("num-envs", int, 10), # Number of parallel envs. 10 by default
    get_arg_dict("num-steps", int, 150), # For each env, how many steps are collected to form PPO Agent rollout.
    get_arg_dict("num-minibatches", int, 1), # Number of mini-batches the rollout data is split into to make the updates
    get_arg_dict("update-epochs", int, 4), # Number of gradient step for the policy and value networks
    get_arg_dict("gamma", float, 0.99),
    get_arg_dict("gae-lambda", float, 0.95),
    get_arg_dict("norm-adv", bool, True, metatype="bool"),
    get_arg_dict("clip-coef", float, 0.1), # Surrogate loss clipping coefficient
    get_arg_dict("clip-vloss", bool, True, metatype="bool"),
    get_arg_dict("ent-coef", float, 0.0), # Entropy loss coef; 0.2 in SS baselines
    get_arg_dict("vf-coef", float, 0.5), # Value loss coefficient
    get_arg_dict("max-grad-norm", float, 0.5),
    get_arg_dict("target-kl", float, None),
    get_arg_dict("lr", float, 2.5e-4), # Learning rate
    get_arg_dict("optim-wd", float, 0), # weight decay for adam optim
    ## Agent network params
    get_arg_dict("agent-type", str, "ss-default", metatype="choice",
        choices=["ss-default", "deep-etho",
                    "perceiver-gwt-gwwm", "perceiver-gwt-attgru"]),
    get_arg_dict("hidden-size", int, 512), # Size of the visual / audio features and RNN hidden states 
    ## Perceiver / PerceiverIO params: TODO: num_latnets, latent_dim, etc...
    get_arg_dict("pgwt-latent-type", str, "randn", metatype="choice",
        choices=["randn", "zeros"]), # Depth of the Perceiver
    get_arg_dict("pgwt-latent-learned", bool, True, metatype="bool"),
    get_arg_dict("pgwt-depth", int, 1), # Depth of the Perceiver
    get_arg_dict("pgwt-num-latents", int, 8),
    get_arg_dict("pgwt-latent-dim", int, 64),
    get_arg_dict("pgwt-cross-heads", int, 1),
    get_arg_dict("pgwt-latent-heads", int, 4),
    get_arg_dict("pgwt-cross-dim-head", int, 64),
    get_arg_dict("pgwt-latent-dim-head", int, 64),
    get_arg_dict("pgwt-weight-tie-layers", bool, False, metatype="bool"),
    get_arg_dict("pgwt-ff", bool, False, metatype="bool"),
    get_arg_dict("pgwt-num-freq-bands", int, 6),
    get_arg_dict("pgwt-max-freq", int, 10.),
    get_arg_dict("pgwt-use-sa", bool, False, metatype="bool"),
    ## Peceiver Modality Embedding related
    get_arg_dict("pgwt-mod-embed", int, 0), # Learnable modality embeddings
    ## Additional modalities
    get_arg_dict("pgwt-ca-prev-latents", bool, False, metatype="bool"), # if True, passes the prev latent to CA as KV input data

    ## Special BC
    get_arg_dict("prev-actions", bool, False, metatype="bool"),
    get_arg_dict("burn-in", int, 0), # Steps used to init the latent state for RNN component
    get_arg_dict("batch-chunk-length", int, 0), # For gradient accumulation
    get_arg_dict("ce-weights", float, None, metatype="list"), # Weights for the Cross Entropy loss
    get_arg_dict("dataset-ce-weights", bool, True, metatype="bool"),

    # Eval protocol
    get_arg_dict("eval", bool, True, metatype="bool"),
    get_arg_dict("eval-every", int, int(1.5e4)), # Every X frames || steps sampled
    get_arg_dict("eval-n-episodes", int, 5),

    # Logging params
    # NOTE: While supported, video logging is expensive because the RGB generation in the
    # envs hogs a lot of GPU, especially with multiple envs 
    get_arg_dict("save-videos", bool, False, metatype="bool"),
    get_arg_dict("save-model", bool, True, metatype="bool"),
    get_arg_dict("log-sampling-stats-every", int, int(1.5e3)), # Every X frames || steps sampled
    get_arg_dict("log-training-stats-every", int, int(10)), # Every X model update
    get_arg_dict("logdir-prefix", str, "./logs/") # Overrides the default one
]
args = generate_args(CUSTOM_ARGS)
# endregion: Generating additional hyparams

# Load environment config
is_SAVi = str.__contains__(args.config_path, "savi")
if is_SAVi:
    env_config = get_savi_config(config_paths=args.config_path)
else:
    env_config = get_config(config_paths=args.config_path)

# Additional PPO overrides
args.batch_size = int(args.num_envs * args.num_steps)
args.minibatch_size = int(args.batch_size // args.num_minibatches)

# Gradient accumulation support
if args.batch_chunk_length == 0:
    args.batch_chunk_length = args.num_envs

# Experiment logger
tblogger = TBLogger(exp_name=args.exp_name, args=args)
print(f"# Logdir: {tblogger.logdir}")
should_log_training_stats = tools.Every(args.log_training_stats_every)
should_eval = tools.Every(args.eval_every)

# Seeding
random.seed(args.seed)
np.random.seed(args.seed)
th.manual_seed(args.seed)
th.cuda.manual_seed_all(args.seed)
th.backends.cudnn.deterministic = args.torch_deterministic
# th.backends.cudnn.benchmark = args.cudnn_benchmark

# Set device as GPU
device = tools.get_device(args) if (not args.cpu and th.cuda.is_available()) else th.device("cpu")

# Overriding some envs parametes from the .yaml env config
env_config.defrost()
# NOTE: using less environments for eval to save up system memory -> run more experiment at thte same time
env_config.NUM_PROCESSES = 2 # Corresponds to number of envs, makes script startup faster for debugs
# env_config.CONTINUOUS = args.env_continuous
## In caes video saving is enabled, make sure there is also the rgb videos
agent_extra_rgb = False
if args.save_videos:
    # For RGB video sensors
    if "RGB_SENSOR" not in env_config.SENSORS:
        env_config.SENSORS.append("RGB_SENSOR")
        # Indicates to the agent that RGB obs should not be used as observational inputs
        agent_extra_rgb = True
    # For Waveform to generate audio over the videos
    if "AUDIOGOAL_SENSOR" not in env_config.TASK_CONFIG.TASK.SENSORS:
        env_config.TASK_CONFIG.TASK.SENSORS.append("AUDIOGOAL_SENSOR")
env_config.freeze()
# print(env_config)

2023-05-22 15:01:42.930752: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


# Logdir: ./logs/_seed_111__2023_05_22_15_01_42_744417.musashi


In [5]:
# Environment instantiation
envs = construct_envs(env_config, get_env_class(env_config.ENV_NAME))
single_observation_space = envs.observation_spaces[0]
single_action_space = envs.action_spaces[0]

# TODO: delete the envrionemtsn / find a more efficient method to do this

# TODO: make the ActorCritic components parameterizable through comand line ?
if args.agent_type == "ss-default":
    agent = ActorCritic(single_observation_space, single_action_space,
        args.hidden_size, extra_rgb=agent_extra_rgb, prev_actions=args.prev_actions).to(device)
elif args.agent_type == "perceiver-gwt-gwwm":
    agent = Perceiver_GWT_GWWM_ActorCritic(single_observation_space, single_action_space,
        args, extra_rgb=agent_extra_rgb).to(device)
elif args.agent_type == "perceiver-gwt-attgru":
    agent = Perceiver_GWT_AttGRU_ActorCritic(single_observation_space, single_action_space,
        args, extra_rgb=agent_extra_rgb).to(device)
elif args.agent_type == "deep-etho":
    raise NotImplementedError(f"Unsupported agent-type:{args.agent_type}")
    # TODO: support for storing the rnn_hidden_statse, so that the policy 
    # that takes in the 'core_modules' 's rnn hidden output can also work.
    agent = ActorCritic_DeepEthologyVirtualRodent(single_observation_space,
            single_action_space, 512).to(device)
else:
    raise NotImplementedError(f"Unsupported agent-type:{args.agent_type}")

if not args.cpu and th.cuda.is_available():
    # TODO: GPU only. But what if we still want to use the default pytorch one ?
    optimizer = apex.optimizers.FusedAdam(agent.parameters(), lr=args.lr, eps=1e-5, weight_decay=args.optim_wd)
else:
    optimizer = th.optim.Adam(agent.parameters(), lr=args.lr, eps=1e-5, weight_decay=args.optim_wd)

optimizer.zero_grad()

2023-05-22 15:01:44,242 Initializing dataset SemanticAudioNav
2023-05-22 15:01:44,452 Initializing dataset SemanticAudioNav
2023-05-22 15:02:05,738 initializing sim SoundSpacesSim
2023-05-22 15:02:05,891 Initializing task SemanticAudioNav
2023-05-22 15:02:05,970 Initializing dataset SemanticAudioNav
2023-05-22 15:02:25,728 initializing sim SoundSpacesSim
2023-05-22 15:02:25,874 Initializing task SemanticAudioNav


In [6]:
# Dataset loading
dloader = make_dataloader3(args.dataset_path, batch_size=args.num_envs,
                            batch_length=args.num_steps, seed=args.seed)

## Compute action coefficient for CEL of BC
dataset_stats_filepath = f"{args.dataset_path}/dataset_statistics.bz2"
# Override dataset statistics if the file already exists
if os.path.exists(dataset_stats_filepath):
    with open(dataset_stats_filepath, "rb") as f:
        dataset_statistics = cpkl.load(f)

# Reads args.ce_weights if passed
ce_weights = args.ce_weights

# In case args.dataset_ce_weights is True,
# override the args.ce_weigths manual setting
if args.dataset_ce_weights:
    # TODO: make some assert on 1) the existence of the "dataset_statistics.bz2" file
    # and 2) that it contains the "action_cel_coefs" of proper dimension
    ce_weights = [dataset_statistics["action_cel_coefs"][a] for a in range(4)]
    print("## Loading CEL weights from dataset: ", ce_weights)

if ce_weights is not None:
    # TODO: assert it has same shape as action space.
    ce_weights = th.Tensor(ce_weights).to(device)
    print("## Manually set CEL weights from dataset: ", ce_weights)

# Info logging
# summary(agent)
# print("")
# print(agent)
# print("")

Initialized IterDset with 26508 episodes.
## Loading CEL weights from dataset:  [4.71576376051609, 0.35938861176199993, 2.2051043449115317, 1.811762617039163]
## Manually set CEL weights from dataset:  tensor([4.7158, 0.3594, 2.2051, 1.8118], device='cuda:0')


In [7]:
ce_weights

tensor([4.7158, 0.3594, 2.2051, 1.8118], device='cuda:0')

In [8]:
obs_list, action_list, _, done_list, depad_mask_list = \
    [ {k: th.Tensor(v).float().to(device) for k,v in b.items()} if isinstance(b, dict) else 
        b.float().to(device) for b in next(dloader)] # NOTE this will not suport "audiogoal" waveform audio, only rgb / depth / spectrogram

Sampled traj idx: 22582; Length: 17; Success: 1.0; Last act: 0
Sampled traj idx: 5217; Length: 24; Success: 1.0; Last act: 0
Sampled traj idx: 9815; Length: 14; Success: 1.0; Last act: 0
Sampled traj idx: 16349; Length: 18; Success: 1.0; Last act: 0
Sampled traj idx: 6750; Length: 13; Success: 1.0; Last act: 0
Sampled traj idx: 12645; Length: 18; Success: 1.0; Last act: 0
Sampled traj idx: 14886; Length: 11; Success: 1.0; Last act: 0
Sampled traj idx: 24855; Length: 17; Success: 1.0; Last act: 0
Sampled traj idx: 4665; Length: 24; Success: 1.0; Last act: 0
Sampled traj idx: 6566; Length: 34; Success: 1.0; Last act: 0
Sampled traj idx: 26198; Length: 11; Success: 1.0; Last act: 0
Sampled traj idx: 20878; Length: 18; Success: 1.0; Last act: 0
Sampled traj idx: 24348; Length: 13; Success: 1.0; Last act: 0
Sampled traj idx: 25648; Length: 14; Success: 1.0; Last act: 0
Sampled traj idx: 17761; Length: 10; Success: 1.0; Last act: 0
Sampled traj idx: 6505; Length: 43; Success: 1.0; Last act: 

In [9]:
# Checking shapes of the sampled batch data, so far so good
# action_list.shape, done_list.shape, depad_mask_list.shape, obs_list["rgb"].shape, obs_list["depth"].shape, obs_list["spectrogram"].shape, obs_list["audiogoal"].shape

In [10]:
# Checking the dataset steps
from pprint import pprint
pprint(dataset_statistics)

{'action_cel_coefs': {0: 4.71576376051609,
                      1: 0.35938861176199993,
                      2: 2.2051043449115317,
                      3: 1.811762617039163},
 'action_counts': {0: 26507, 1: 347815, 2: 56687, 3: 68994},
 'action_probs': {0: 0.05301368191790849,
                  1: 0.6956258262450425,
                  2: 0.11337331976008144,
                  3: 0.13798717207696753},
 'category_counts': {'bathtub': 78,
                     'bed': 645,
                     'cabinet': 2388,
                     'chair': 6903,
                     'chest_of_drawers': 690,
                     'clothes': 117,
                     'counter': 849,
                     'cushion': 1821,
                     'fireplace': 231,
                     'gym_equipment': 79,
                     'picture': 3563,
                     'plant': 1400,
                     'seating': 813,
                     'shower': 230,
                     'sink': 807,
                     'sofa': 

In [11]:
# Training start
start_time = time.time()
num_updates = args.total_steps // args.batch_size # Total number of updates that will take place in this experiment
n_updates = 0 # Progressively tracks the number of network updats

# NOTE: 10 * 150 as step to match the training rate of an RL Agent, irrespective of which batch size / batch length is used
for global_step in range(1, args.total_steps + 1, 10 * 150):
    # Load batch data
    obs_list, action_list, _, done_list, depad_mask_list = \
        [ {k: th.Tensor(v).float().to(device) for k,v in b.items()} if isinstance(b, dict) else 
            b.float().to(device) for b in next(dloader)] # NOTE this will not suport "audiogoal" waveform audio, only rgb / depth / spectrogram
    
    # NOTE: RGB are normalized in the VisualCNN module
    # PPO networks expect input of shape T,B, ... so doing the permutation here
    for k, v in obs_list.items():
        if k in ["rgb", "spectrogram", "depth"]:
            obs_list[k] = v.permute(1, 0, 2, 3, 4) # BTCHW -> TBCHW
        elif k in ["audiogoal"]:
            obs_list[k] = v.permute(1, 0, 2, 3) # BTCL -> TBC
        else:
            pass
    
    action_list = action_list.permute(1, 0, 2)
    done_list = done_list.permute(1, 0, 2)
    depad_mask_list = depad_mask_list.permute(1, 0, 2)

    prev_actions_list = th.zeros_like(action_list)
    prev_actions_list[1:] = action_list[:-1]
    prev_actions_list = F.one_hot(prev_actions_list.long()[:, :, 0], num_classes=4).float()
    prev_actions_list[0] = prev_actions_list[0] * 0.

    # PPO Update Phase: actor and critic network updates
    # assert args.num_envs % args.num_minibatches == 0
    # envsperbatch = args.num_envs // args.num_minibatches
    # envinds = np.arange(args.num_envs)
    # flatinds = np.arange(args.batch_size).reshape(args.num_envs, args.num_steps)

    for _ in range(args.update_epochs):
        # np.random.shuffle(envinds)
        # TODO / MISSING: mini-batch updates support like in ppo_av_nav.py
        assert args.num_envs % args.batch_chunk_length == 0, \
            f"num-envs (batch-size) of {args.num_envs} should be integer divisible by {args.batch_chunk_length}"
        
        # For gradient accumulation over large batches
        n_bchunks = args.num_envs // args.batch_chunk_length
        
        # Reset optimizer for each chunk over the "trajectory length" axis
        optimizer.zero_grad()

        # Placeholder for tracking the actions of the agent
        batch_traj_agent_actions = th.zeros_like(action_list).long()

        for bchnk_idx in range(n_bchunks):
            # This will be used to recompute the rnn_hidden_states when computiong the new action logprobs
            if args.agent_type == "ss-default":
                rnn_hidden_state = th.zeros((1, args.batch_chunk_length, args.hidden_size), device=device)
            elif args.agent_type in ["perceiver-gwt-gwwm", "perceiver-gwt-attgru"]:
                rnn_hidden_state = agent.state_encoder.latents.repeat(args.batch_chunk_length, 1, 1)
            else:
                raise NotImplementedError(f"Unsupported agent-type:{args.agent_type}")

            b_chnk_start = bchnk_idx * args.batch_chunk_length
            b_chnk_end = (bchnk_idx + 1) * args.batch_chunk_length

            # NOTE: v.shape[-3:] only valid for "rgb", "depth", and "spectrogram"
            obs_chunk_list = {}
            for k,v in obs_list.items():
                if k in ["rgb", "depth", "spectrogram"]:
                    reshaped_v = v[:, b_chnk_start:b_chnk_end].reshape(-1, *v.shape[-3:])
                elif k in ["audiogoal"]:
                    reshaped_v = v[:, b_chnk_start:b_chnk_end].reshape(-1, *v.shape[-2:])
                else:
                    reshaped_v = v[:, b_chnk_start:b_chnk_end].reshape(-1, *v.shape[-1:])
                
                obs_chunk_list[k] = reshaped_v

            # obs_chunk_list = {k: v[:, b_chnk_start:b_chnk_end].reshape(-1, *v.shape[(-3 if k in ["rgb", "depth", "spectrogram"] else -2):])
            #                     for k, v in obs_list.items()}
            masks_chunk_list = 1. - done_list[:, b_chnk_start:b_chnk_end].reshape(-1, 1)
            action_target_chunk_list = action_list[:, b_chnk_start:b_chnk_end, 0].reshape(-1).long()
            prev_actions_chunk_list = prev_actions_list[:, b_chnk_start:b_chnk_end].reshape(-1, 4)

            # TODO: maybe detach the rnn_hidden_state between two chunks ?
            actions, action_probs, _, entropies, _, _ = \
                agent.act(obs_chunk_list, rnn_hidden_state,
                    masks=masks_chunk_list) #, prev_actions=prev_actions_chunk_list)
            

            bc_loss = F.cross_entropy(action_probs, action_target_chunk_list,
                                        weight=ce_weights, reduction="none")
            bc_loss = th.masked_select(
                bc_loss,
                depad_mask_list[:, b_chnk_start:b_chnk_end, 0].reshape(-1).bool()
            ).mean()
            
            bc_loss /= n_bchunks # Normalize accumulated grads over batch axis
            
            # Entropy loss
            # TODO: consider making this decaying as training progresses
            entropy = th.masked_select(
                entropies,
                depad_mask_list[:, b_chnk_start:b_chnk_end, 0].reshape(-1).bool()
            ).mean()
            entropy /= n_bchunks # Normalize accumulated grads over batch axis

            # Backpropagate and accumulate gradients over the batch size axis
            (bc_loss - args.ent_coef * entropy).backward()

            # Temporarily save the batch chunk actions of the agent for statistics later
            batch_traj_agent_actions[:, b_chnk_start:b_chnk_end, :] = actions.detach().view(args.num_steps, args.batch_chunk_length, 1)

        grad_norms_preclip = agent.get_grad_norms()
        if args.max_grad_norm > 0:
            nn.utils.clip_grad_norm_(agent.parameters(), args.max_grad_norm)
        optimizer.step()

        n_updates += 1

    if n_updates > 0 and should_log_training_stats(n_updates):
        print(f"Step {global_step} / {args.total_steps}")

        # TODO: add entropy logging
        train_stats = {
            "bc_loss": bc_loss.item() * n_bchunks,
            "entropy": entropy.item() * n_bchunks
        } # * n_bchunks undoes the scaling applied during grad accum
        
        tblogger.log_stats(train_stats, global_step, prefix="train")
    
        # TODO: Additional dbg stats; add if needed
        # debug_stats = {
        #     # Additional debug stats
        #     "b_state_feats_avg": b_state_feats.flatten(start_dim=1).mean().item(),
        #     "init_rnn_states_avg": init_rnn_state.flatten(start_dim=1).mean().item(),
        # }
        # if args.pgwt_mod_embed:
        #     debug_stats["mod_embed_avg"] = agent.state_encoder.modality_embeddings.mean().item()

        # Tracking stats about the actions distribution in the batches # TODO: fix typo
        batch_traj_final_step_idxs = (depad_mask_list.sum(dim=0)-1)[None, :, :].long().reshape(-1).tolist()
        batch_traj_final_step_mask = th.zeros_like(depad_mask_list).long()
        for bi, done_t in enumerate(batch_traj_final_step_idxs):
            batch_traj_final_step_mask[done_t, bi, 0] = 1

        batch_traj_final_actions = th.masked_select(
            action_list,
            batch_traj_final_step_mask.bool()
        )
        # How many '0' actions are sampled in a batch ?
        n_zero_batch_final_actions = len(th.where(batch_traj_final_actions == 0)[0])
        # Ratio of 0 to other actions in the batch
        n_zero_batch_final_actions_ratio = n_zero_batch_final_actions / depad_mask_list.sum().item()


        batch_traj_agent_final_actions = th.masked_select(
            batch_traj_agent_actions,
            batch_traj_final_step_mask.bool()
        )
        # How many '0' actions sampled by the agent given the observations ?
        n_zero_agent_final_actions = len(th.where(batch_traj_agent_final_actions == 0)[0])
        # Ratio of 0 to other actions in the batch
        n_zero_agent_final_actions_ratio = n_zero_agent_final_actions / depad_mask_list.sum().item()

        # Special: tracking the 'StOP' action stats across in the batch
        tblogger.log_stats({
            "n_zero_batch_final_actions": n_zero_batch_final_actions,
            "n_zero_batch_final_actions_ratio": n_zero_batch_final_actions_ratio,
            "n_zero_agent_final_actions": n_zero_agent_final_actions,
            "n_zero_agent_final_actions_ratio": n_zero_agent_final_actions_ratio
        }, step=global_step, prefix="metrics")

        # Logging grad norms
        tblogger.log_stats(agent.get_grad_norms(), global_step, prefix="debug/grad_norms")
        tblogger.log_stats(grad_norms_preclip, global_step, prefix="debug/grad_norms_preclip")

        info_stats = {
            "global_step": global_step,
            "duration": time.time() - start_time,
            "fps": tblogger.track_duration("fps", global_step),
            "n_updates": n_updates,

            "env_step_duration": tblogger.track_duration("fps_inv", global_step, inverse=True),
            "model_updates_per_sec": tblogger.track_duration("model_updates",
                n_updates),
            "model_update_step_duration": tblogger.track_duration("model_updates_inv",
                n_updates, inverse=True),
            "batch_real_steps_ratio": depad_mask_list.sum().item() / np.prod(depad_mask_list.shape)
        }
        tblogger.log_stats(info_stats, global_step, "info")

    # if args.eval and should_eval(global_step):
    #     eval_window_episode_stas = eval_agent(args, envs, agent,
    #         device=device, tblogger=tblogger,
    #         env_config=env_config, current_step=global_step,
    #         n_episodes=args.eval_n_episodes, save_videos=True,
    #         is_SAVi=is_SAVi)
    #     episode_stats = {k: np.mean(v) for k, v in eval_window_episode_stas.items()}
    #     episode_stats["last_actions_min"] = np.min(eval_window_episode_stas["last_actions"])
    #     tblogger.log_stats(episode_stats, global_step, "metrics")
    
    #     if args.save_model:
    #         model_save_dir = tblogger.get_models_savedir()
    #         model_save_name = f"ppo_agent.{global_step}.ckpt.pth"
    #         model_save_fullpath = os.path.join(model_save_dir, model_save_name)

    #         th.save(agent.state_dict(), model_save_fullpath)

# Clean up
tblogger.close() 
envs.close()

Step 1 / 1000000
Step 4501 / 1000000
Step 7501 / 1000000
Step 12001 / 1000000
Step 15001 / 1000000
Step 19501 / 1000000
Step 22501 / 1000000
Step 27001 / 1000000
Step 30001 / 1000000
Step 34501 / 1000000


EOFError: Caught EOFError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/rousslan/anaconda3/envs/ss-hab-headless-py39/lib/python3.9/site-packages/torch/utils/data/_utils/worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/rousslan/anaconda3/envs/ss-hab-headless-py39/lib/python3.9/site-packages/torch/utils/data/_utils/fetch.py", line 32, in fetch
    data.append(next(self.dataset_iter))
  File "/tmp/ipykernel_21756/998088775.py", line 24, in __iter__
    edd = cpkl.load(f)
  File "/home/rousslan/anaconda3/envs/ss-hab-headless-py39/lib/python3.9/site-packages/compress_pickle/compress_pickle.py", line 272, in load
    output = uncompress_and_unpickle(
  File "/home/rousslan/anaconda3/envs/ss-hab-headless-py39/lib/python3.9/functools.py", line 888, in wrapper
    return dispatch(args[0].__class__)(*args, **kw)
  File "/home/rousslan/anaconda3/envs/ss-hab-headless-py39/lib/python3.9/site-packages/compress_pickle/io/b